In [1]:
import pandas as pd
import nltk
data = pd.read_csv("hoi_dap_vui.csv")
khach_hang = data["user_a"].to_list()
nhan_vien = data["user_b"].to_list()
def chat_bot_blue(cau_hoi):
  diem_tuong_dong = 0
  cau_tra_Loi = ""
  for i in range(len(khach_hang)):
    BLEU_score = nltk.translate.bleu_score.sentence_bleu([cau_hoi], khach_hang[i], weights = (0.5, 0.5))
    if BLEU_score > diem_tuong_dong:
      diem_tuong_dong = BLEU_score
      cau_tra_Loi = nhan_vien [i]
  return cau_tra_Loi
print(chat_bot_blue("Bạn có bắt chuyện trước không"))

C:\Users\PC\AppData\Local\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Có đôi khi


In [2]:
import pandas as pd
from gensim.models import Word2Vec
import re
import numpy as np 
import tensorflow as tf
import pickle
from tensorflow.keras import layers, activations, models, preprocessing, utils
questions = data["user_a"].tolist()
answers = data["user_b"].tolist()
answers_with_tags = list()

In [3]:
# Xóa đi cột bị rỗng
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )
answers = list()


In [4]:
#gan nhan 
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )
print(answers[0])

<START> Dễ thương, tóc dài, da trắng <END>


In [5]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'Số lượng từ trong từ điển: {}'.format( VOCAB_SIZE ))

Số lượng từ trong từ điển: 3097


In [18]:
tokenizer.word_index

{'end': 1,
 'start': 2,
 'không': 3,
 'có': 4,
 'gì': 5,
 'đi': 6,
 'mày': 7,
 'bạn': 8,
 'ở': 9,
 'đâu': 10,
 'nào': 11,
 'làm': 12,
 'thích': 13,
 'học': 14,
 'bao': 15,
 'người': 16,
 'chơi': 17,
 'là': 18,
 'ăn': 19,
 'nhiêu': 20,
 'chưa': 21,
 'hay': 22,
 'tao': 23,
 'yêu': 24,
 'quê': 25,
 'rồi': 26,
 'nhà': 27,
 'được': 28,
 'với': 29,
 'sao': 30,
 'trường': 31,
 'phim': 32,
 'đang': 33,
 'tên': 34,
 'mua': 35,
 'ai': 36,
 'mấy': 37,
 'thế': 38,
 'tết': 39,
 'về': 40,
 'mình': 41,
 'xem': 42,
 'năm': 43,
 'vậy': 44,
 'cũng': 45,
 'tuổi': 46,
 'muốn': 47,
 'thì': 48,
 '1': 49,
 'này': 50,
 'của': 51,
 'nhiều': 52,
 'công': 53,
 'như': 54,
 'em': 55,
 'đó': 56,
 'ngày': 57,
 'biết': 58,
 'thể': 59,
 'và': 60,
 'còn': 61,
 'khi': 62,
 'ok': 63,
 '2': 64,
 'đẹp': 65,
 'thường': 66,
 'vui': 67,
 'chứ': 68,
 'nhất': 69,
 'sinh': 70,
 'anh': 71,
 'à': 72,
 'cho': 73,
 'con': 74,
 'việc': 75,
 'nước': 76,
 'lắm': 77,
 'game': 78,
 'lâu': 79,
 'một': 80,
 'cao': 81,
 'tính': 82,
 '3': 83

In [16]:
#trích thành các từ xl các kí tự đặc biệt
vocab = []
for word in tokenizer.word_index:
  vocab.append(word)

def tokenize(sentences):
  tokens_list = []
  vocabulary = []
  for sentence in sentences:
    sentence = sentence.lower()
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    tokens = sentence.split()
    vocabulary += tokens
    tokens_list.append(tokens)
  return tokens_list, vocabulary


In [7]:
#encoder_input_data mã hóa câu hỏi thành vector số(one hot vector)
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post')
encoder_input_data = np.array(padded_questions)
print()
print("Encoder_input_data",encoder_input_data.shape, maxlen_questions)
print(questions[0])
print(encoder_input_data[0])


Encoder_input_data (5899, 93) 93
Thích mẫu người nào
[ 13 344  16  11   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0]


In [8]:
# decoder_input_data mã hóa câu trả lời thành vector số
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print(  )
print("Decoder_input_data",decoder_input_data.shape , maxlen_answers)
print(answers[0])
print(decoder_input_data[0])


Decoder_input_data (5899, 62) 62
<START> Dễ thương, tóc dài, da trắng <END>
[   2  160  306 1645  307 2261  603    1    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]


In [9]:
# decoder_output_data giải mã từ vector số sang string
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print(  )
print("Decoder_output_data", decoder_output_data.shape)
print(answers[0])
print(decoder_output_data[0])


Decoder_output_data (5899, 62, 3097)
<START> Dễ thương, tóc dài, da trắng <END>
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [10]:
#khởi tạo mô hình lstm
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

# model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size= 10, epochs=1 ) 



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 93)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 62)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 93, 200)              619400    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 62, 200)              619400    ['input_2[0][0]']             
                                                                                             

In [12]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size= 10, epochs=2 ) 

Epoch 1/2
590/590 [==============================] - 133s 225ms/step - loss: 4.2115
Epoch 2/2
590/590 [==============================] - 140s 237ms/step - loss: 4.1108


In [ ]:
model.save('model1.h5')

In [11]:
model.load_weights("model1.h5")

In [14]:
#ham thu mo hinh ,nhận đàu vào là 1 câu hỏi, đầu ra: câu trả lời
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model
    
def str_to_tokens( sentence : str ):

    words = sentence.lower().split()
    tokens_list = list()
  
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

enc_model , dec_model = make_inference_models()

def chat_bot_lstm(cau_hoi):
    states_values = enc_model.predict( str_to_tokens( cau_hoi ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 
        print(decoded_translation)
    return ( decoded_translation[1:-4])


In [16]:
print(chat_bot_lstm("quê bạn ở đâu"))

1/1 [==============================] - 0s 31ms/step
 sài
1/1 [==============================] - 0s 39ms/step
 sài gòn
1/1 [==============================] - 0s 42ms/step
 sài gòn end
sài gòn


In [17]:
questions
answers
BLEU_score = 0
for i in range(100):
    try:
        BLEU_score += nltk.translate.bleu_score.sentence_bleu([answers[i]],chat_bot_lstm(questions[i]), weights = (0.5, 0.5))
    except: 
        BLEU_score += 1
print("Điểm số đánh giá BLEU score đạt được",BLEU_score/100)

1/1 [==============================] - 0s 39ms/step
 người
1/1 [==============================] - 0s 39ms/step
 người yêu
1/1 [==============================] - 0s 46ms/step
 người yêu quận
1/1 [==============================] - 0s 46ms/step
 người yêu quận 7
1/1 [==============================] - 0s 48ms/step
 người yêu quận 7 end
1/1 [==============================] - 0s 36ms/step
 có
1/1 [==============================] - 0s 47ms/step
 có end
1/1 [==============================] - 0s 62ms/step


C:\Users\PC\AppData\Local\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


1/1 [==============================] - 0s 42ms/step
 vì
1/1 [==============================] - 0s 40ms/step
 vì vì
1/1 [==============================] - 0s 55ms/step
 vì vì tao
1/1 [==============================] - 0s 56ms/step
 vì vì tao thích
1/1 [==============================] - 0s 34ms/step
 vì vì tao thích bạn
1/1 [==============================] - 0s 39ms/step
 vì vì tao thích bạn end
1/1 [==============================] - 0s 31ms/step
 có
1/1 [==============================] - 0s 30ms/step
 có end
1/1 [==============================] - 0s 32ms/step
 không
1/1 [==============================] - 0s 30ms/step
 không end
1/1 [==============================] - 0s 33ms/step
 có
1/1 [==============================] - 0s 36ms/step
 có nhiều
1/1 [==============================] - 0s 34ms/step
 có nhiều end
1/1 [==============================] - 0s 32ms/step
 cũng
1/1 [==============================] - 0s 34ms/step
 cũng hay
1/1 [==============================] - 0s 39ms/step
 cũng hay